In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.optimize import minimize, differential_evolution, bisect
from scipy.stats import norm, multivariate_normal
from scipy.special import gammaln
from scipy.special import k1e
from scipy.integrate import quad
from copulae import GaussianCopula

In [23]:
# load data

df_portfolio = pd.read_csv('initial_portfolio.csv')
df_rf = pd.read_csv('rf.csv')
df_prices = pd.read_csv('DailyPrices.csv')

df_prices['Date'] = pd.to_datetime(df_prices['Date'])
df_rf['Date'] = pd.to_datetime(df_rf['Date'])
df_prices.set_index('Date', inplace=True)
df_rf.set_index('Date', inplace=True)

df_returns = df_prices.pct_change().dropna()
train_end = pd.Timestamp('2023-12-31')
train_returns = df_returns[df_returns.index <= train_end]
hold_returns = df_returns[df_returns.index > train_end]

train_rf = df_rf[df_rf.index <= train_end]
hold_rf = df_rf[df_rf.index > train_end]

In [24]:
display(train_returns)

,SPY,AAPL,NVDA,MSFT,AMZN,META,GOOGL,AVGO,TSLA,GOOG,...,SBUX,MMC,MDT,CB,LMT,KKR,MU,PLD,LRCX,EQIX
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-04,0.007720,0.010314,0.030318,-0.043743,-0.007924,0.021084,-0.011670,0.012214,0.051249,-0.011037,...,0.036001,0.019460,0.034628,0.016442,-0.002157,0.030420,0.076037,0.037892,0.019696,0.026626
2023-01-05,-0.011413,-0.010605,-0.032816,-0.029638,-0.023726,-0.003376,-0.021344,-0.009318,-0.029039,-0.021869,...,-0.000287,-0.018143,-0.011609,-0.003743,0.001196,-0.014553,0.009410,-0.035140,-0.012782,-0.028714
2023-01-06,0.022932,0.036794,0.041640,0.011785,0.035611,0.024263,0.013225,0.060196,0.024651,0.016019,...,0.021641,0.029012,0.010371,0.023707,-0.008028,0.016456,0.037653,0.033673,0.067640,0.020163
2023-01-09,-0.000567,0.004089,0.051753,0.009736,0.014870,-0.004230,0.007786,-0.019612,0.059349,0.007260,...,-0.018277,-0.003334,-0.041059,-0.023377,-0.030111,0.030303,-0.007222,-0.005058,0.016080,0.010713
2023-01-10,0.007013,0.004456,0.017981,0.007617,0.028732,0.027188,0.004544,-0.003398,-0.007681,0.004955,...,0.012030,0.000000,0.017410,0.005637,0.007190,0.009871,0.015082,-0.000086,0.013660,0.020539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,0.002010,-0.005547,-0.003266,0.002784,-0.002730,-0.001977,0.007620,-0.004710,-0.007701,0.006488,...,-0.000524,0.005781,-0.000488,0.003409,0.000446,0.000855,0.011816,0.010996,0.003952,-0.002395
2023-12-26,0.004223,-0.002841,0.009195,0.000214,-0.000065,0.004075,0.000212,0.008833,0.016116,0.000701,...,0.004093,-0.002129,0.004639,0.005889,0.004373,0.003174,0.006590,0.008006,0.020154,0.005577
2023-12-27,0.001808,0.000518,0.002800,-0.001575,-0.000456,0.008455,-0.008126,-0.005054,0.018822,-0.009662,...,-0.003972,0.005387,0.001458,0.002792,-0.002688,0.014115,-0.004594,0.005395,-0.000980,0.003395


**Part 1**

In [25]:
# fit capm model
stocks = df_portfolio['Symbol'].unique().tolist()
n_stocks = len(stocks)


capm_params = {}
betas = []

for ticker in stocks:
    if ticker == "SPY":
        capm_params[ticker] = {'alpha': 0, 'beta': 1, 'residuals': 0}
        betas.append(1)
        continue
    
    stock_train = train_returns[ticker].dropna()
    df_temp = pd.DataFrame({
        'Stock': stock_train,
        'market': train_returns['SPY'],
        'rf': train_rf['rf']
    }).dropna()
        
    df_temp['excess_stock'] = df_temp['Stock'] - df_temp['rf']
    df_temp['excess_market'] = df_temp['market'] - df_temp['rf']
        
    X = sm.add_constant(df_temp['excess_market'])
    y = df_temp['excess_stock']
    model = sm.OLS(y, X).fit()
    beta_value = model.params['excess_market']
    capm_params[ticker] = {
        'alpha': model.params['const'],
        'beta': beta_value,
        'residuals': model.mse_resid
    }
    betas.append(beta_value)
    
betas = np.array(betas)
capm_betas = pd.DataFrame({'Symbol': stocks, 'Beta': betas})
print(capm_betas)

   Symbol      Beta
0     WFC  1.140628
1     ETN  1.116652
2    AMZN  1.532365
3    QCOM  1.479601
4     LMT  0.320696
..    ...       ...
94   MSFT  1.169683
95    PEP  0.376748
96     CB  0.459826
97   PANW  1.172476
98    BLK  1.243292

[99 rows x 2 columns]


In [100]:
realized_returns = hold_returns[stocks]
realized_spy = pd.DataFrame({'SPY': hold_returns['SPY']})
last_date = train_returns.index[-1]
start_prices = df_prices.loc[last_date, stocks]

In [101]:

# expost factor attribution
def expost_factor(w, upReturns, upFfData, Betas):
    stocks = upReturns.columns.tolist()
    factors = upFfData.columns.tolist()
    
    n = len(upReturns)
    pReturn = np.zeros(n)
    residReturn = np.zeros(n)
    weights = np.zeros((n, len(w)))
    factorWeights = np.zeros((n, len(factors)))
    lastW = w.values.copy() if isinstance(w, pd.Series) else w.copy()
    
    matReturns = upReturns.values
    ffReturns = upFfData.values
    
    Betas_matrix = np.array(Betas).reshape(-1, 1)
    residIndivual = matReturns - np.dot(ffReturns, Betas_matrix.T)

    for i in range(n):
        weights[i, :] = lastW
        factorWeights[i, :] = np.sum(Betas * lastW)
        lastW = lastW * (1.0 + matReturns[i, :])
        pR = np.sum(lastW)
        lastW = lastW / pR
        pReturn[i] = pR - 1
        residReturn[i] = (pR - 1) - np.dot(factorWeights[i, :], ffReturns[i, :])
    
    upFfData_copy = upFfData.copy()
    upFfData_copy['Alpha'] = residReturn
    upFfData_copy['Portfolio'] = pReturn
    
    totalRet = np.exp(np.sum(np.log(pReturn + 1))) - 1
    
    k = np.log(totalRet + 1) / totalRet if totalRet != 0 else 1
    
    carinoK = np.zeros_like(pReturn)
    non_zero_mask = pReturn != 0
    carinoK[non_zero_mask] = np.log(1.0 + pReturn[non_zero_mask]) / pReturn[non_zero_mask] / k
    carinoK[~non_zero_mask] = 1 

    attrib = pd.DataFrame(
        ffReturns * factorWeights * carinoK.reshape(-1, 1), 
        columns=factors
    )
    attrib['Alpha'] = residReturn * carinoK
    
    for i in range(n):
        residIndivual[i, :] *= weights[i, :]
    

    result = {
        'totalRet': totalRet,
        'attrib': attrib,
        'upFfData': upFfData_copy,
        'weights': weights,
        'factorWeights': factorWeights,
        'residIndivual': residIndivual,
        'residReturn': residReturn,
        'carinoK': carinoK,
        'pReturn': pReturn
    }
    
    return result

In [102]:

def run_attribution(realized_returns, realized_spy, last_date, start_prices, portfolio, betas):
    portfolio_holdings = portfolio.set_index('Symbol')['Holding']
    t_value = (start_prices * portfolio_holdings).sum()
    
    w = pd.Series(0, index=stocks)
    for ticker in stocks:
        if ticker in portfolio_holdings.index:
            w[ticker] = start_prices[ticker] * portfolio_holdings[ticker] / t_value
        else:
            w[ticker] = 0
    
    result = expost_factor(w, realized_returns, realized_spy, betas)
    
    totalRet = result['totalRet']
    attrib = result['attrib']
    upFfData = result['upFfData']
    pReturn = result['pReturn']
    
    attribution = pd.DataFrame({'Value': ['TotalReturn', 'Return Attribution']})

    factors = realized_spy.columns.tolist()
    newFactors = factors + ['Alpha']
    
    for s in newFactors + ['Portfolio']:
        if s in upFfData.columns:
            tr = np.exp(np.sum(np.log(1 + upFfData[s].values))) - 1
            if s != 'Portfolio':
                atr = attrib[s].sum()
            else:
                atr = tr    
            attribution[s] = [tr, atr]
    
    Y = np.column_stack([
        realized_spy.values * result['factorWeights'], 
        result['residReturn']
    ])
    
    X = np.column_stack([np.ones(len(pReturn)), pReturn])
    
    B = np.linalg.inv(X.T @ X) @ X.T @ Y
    betas_vol = B[1, :]
    
    cSD = betas_vol * np.std(pReturn)
    
    vol_attribution = pd.DataFrame({'Value': ['Vol Attribution']})
    for i, factor in enumerate(newFactors):
        vol_attribution[factor] = [cSD[i]]
    
    vol_attribution['Portfolio'] = [np.std(pReturn)]
    
    attribution = pd.concat([attribution, vol_attribution], ignore_index=True)

    print(attribution)
    
    portfolios = portfolio['Portfolio'].unique()
    portfolio_results = []
    
    for p in portfolios:
        print(f"\n{p} portfolio attribution:")
        p_stocks = portfolio[portfolio['Portfolio'] == p]['Symbol'].tolist()
        p_holdings = portfolio[portfolio['Portfolio'] == p].set_index('Symbol')['Holding']
        
        p_t_value = (start_prices[p_stocks] * p_holdings).sum()
        p_w = pd.Series(0, index=stocks)
        
        for ticker in p_stocks:
            p_w[ticker] = start_prices[ticker] * p_holdings[ticker] / p_t_value
        
        p_result = expost_factor(p_w, realized_returns, realized_spy, betas)
        p_totalRet = p_result['totalRet']
        p_attrib = p_result['attrib']
        p_upFfData = p_result['upFfData']
        p_pReturn = p_result['pReturn']
        
        p_attribution = pd.DataFrame({'Value': ['TotalReturn', 'Return Attribution']})
        
        for s in newFactors + ['Portfolio']:
            if s in p_upFfData.columns:

                tr = np.exp(np.sum(np.log(1 + p_upFfData[s].values))) - 1
                
                if s != 'Portfolio':
                    atr = p_attrib[s].sum()
                else:
                    atr = tr
                
                p_attribution[s] = [tr, atr]
        
        p_Y = np.column_stack([
            realized_spy.values * p_result['factorWeights'], 
            p_result['residReturn']
        ])
        
        p_X = np.column_stack([np.ones(len(p_pReturn)), p_pReturn])
        p_B = np.linalg.inv(p_X.T @ p_X) @ p_X.T @ p_Y
        p_betas_vol = p_B[1, :]
        p_cSD = p_betas_vol * np.std(p_pReturn)
        
        p_vol_attribution = pd.DataFrame({'Value': ['Vol Attribution']})
        
        for i, factor in enumerate(newFactors):
            p_vol_attribution[factor] = [p_cSD[i]]
        
        p_vol_attribution['Portfolio'] = [np.std(p_pReturn)]
        
        p_attribution = pd.concat([p_attribution, p_vol_attribution], ignore_index=True)
        
        print(p_attribution)
        
        portfolio_results.append({
            'Portfolio': p,
            'Attribution': p_attribution
        })
    
    return {'Total': attribution, 'Portfolios': portfolio_results}


In [103]:
attribution_results = run_attribution(realized_returns, realized_spy, last_date, start_prices, df_portfolio, betas)


                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373 -0.035969   0.204731
1  Return Attribution  0.244039 -0.039309   0.204731
2     Vol Attribution  0.007207 -0.000131   0.007076

A portfolio attribution:
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373 -0.095555   0.136642
1  Return Attribution  0.242621 -0.105980   0.136642
2     Vol Attribution  0.007056  0.000348   0.007404

B portfolio attribution:
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373 -0.028626   0.203526
1  Return Attribution  0.234259 -0.030733   0.203526
2     Vol Attribution  0.006411  0.000442   0.006854

C portfolio attribution:
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373  0.022337   0.281172
1  Return Attribution  0.255627  0.025546   0.281172
2     Vol Attribution  0.007230  0.000678   0.007908


/var/folders/t0/g3jjcv090sj4mcczh5q9hw040000gn/T/ipykernel_60531/4223567146.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.00806841763974875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  w[ticker] = start_prices[ticker] * portfolio_holdings[ticker] / t_value
/var/folders/t0/g3jjcv090sj4mcczh5q9hw040000gn/T/ipykernel_60531/4223567146.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.02304751711388518' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  p_w[ticker] = start_prices[ticker] * p_holdings[ticker] / p_t_value
/var/folders/t0/g3jjcv090sj4mcczh5q9hw040000gn/T/ipykernel_60531/4223567146.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.022

**Part 2**

In [36]:
# calculate expected returns based on zero alpha assumption
avg_spy_return = train_returns['SPY'].mean()
avg_rf_rate = train_rf['rf'].mean()

expected_excess_returns = {}
for ticker in stocks:
    if ticker == "SPY":
        expected_excess_returns[ticker] = avg_spy_return - avg_rf_rate
    else:
        beta = capm_params[ticker]['beta']
        expected_excess_returns[ticker] = beta * (avg_spy_return - avg_rf_rate)

expected_returns = pd.Series(expected_excess_returns) + avg_rf_rate
cov_matrix = train_returns[stocks].cov()


In [38]:

# portfolio optimization
def negative_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate):
    port_return = np.sum(expected_returns * weights)
    port_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = (port_return - risk_free_rate) / port_stddev
    return -sharpe

def optimize_portfolio(expected_returns, cov_matrix, risk_free_rate, tickers):
    sub_expected_returns = expected_returns[tickers]
    sub_cov_matrix = cov_matrix.loc[tickers, tickers]
    
    initial_weights = np.ones(len(tickers)) / len(tickers)
    bounds = tuple((0, 1) for _ in range(len(tickers)))
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
    
    result = minimize(
        negative_sharpe_ratio,
        initial_weights,
        args=(sub_expected_returns, sub_cov_matrix, risk_free_rate),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    return pd.Series(result['x'], index=tickers)

In [83]:
optimized_portfolios = {}
portfolios = df_portfolio['Portfolio'].unique()

for p in portfolios:
    p_stocks = df_portfolio[df_portfolio['Portfolio'] == p]['Symbol'].tolist()
    
    optimized_weights = optimize_portfolio(expected_returns, cov_matrix, avg_rf_rate, p_stocks)
    
    optimized_portfolios[p] = optimized_weights
    
    port_return = np.sum(expected_returns[p_stocks] * optimized_weights)
    port_stddev = np.sqrt(np.dot(optimized_weights.T, np.dot(cov_matrix.loc[p_stocks, p_stocks], optimized_weights)))
    sharpe = (port_return - avg_rf_rate) * np.sqrt(252) / port_stddev
    
    print(f"\nportfolio {p} after optimized:")
    print(f"expected return: {port_return*252:.6f}")
    print(f"expected vol: {port_stddev * np.sqrt(252):.6f}")
    print(f"Sharpe ratio(annual): {sharpe:.6f}")
    
    original_weights = {}
    p_holdings = df_portfolio[df_portfolio['Portfolio'] == p].set_index('Symbol')['Holding']
    p_t_value = (start_prices[p_stocks] * p_holdings).sum()
    
    for ticker in p_stocks:
        original_weights[ticker] = start_prices[ticker] * p_holdings[ticker] / p_t_value
    
    weight_comparison = pd.DataFrame({
        'Original Weight': pd.Series(original_weights),
        'Optimized Weight': pd.Series(optimized_weights)
    })
    
    print("\nweight comparison:")
    display(weight_comparison)


portfolio A after optimized:
expected return: 0.250315
expected vol: 0.137065
Sharpe ratio(annual): 1.463484

weight comparison:


,Original Weight,Optimized Weight
WFC,0.023048,1.795470e-02
ETN,0.024155,3.816275e-02
AMZN,0.023657,9.117423e-02
QCOM,0.030724,1.089794e-02
LMT,0.031591,2.760596e-02
KO,0.031983,5.694559e-02
JNJ,0.036804,2.373200e-02
ISRG,0.021696,4.279053e-02
XOM,0.031014,0.000000e+00
MDT,0.033995,0.000000e+00



portfolio B after optimized:
expected return: 0.249576
expected vol: 0.134705
Sharpe ratio(annual): 1.483631

weight comparison:


,Original Weight,Optimized Weight
AXP,0.022354,0.021682
HON,0.033054,0.051079
META,0.021340,0.031866
NFLX,0.020799,0.015551
PGR,0.023664,0.000000
LLY,0.026846,0.020992
JPM,0.024840,0.020475
VRTX,0.036212,0.016525
TJX,0.027755,0.055890
EQIX,0.031057,0.035289



portfolio C after optimized:
expected return: 0.251673
expected vol: 0.136208
Sharpe ratio(annual): 1.482658

weight comparison:


,Original Weight,Optimized Weight
IBM,0.027348,0.011916
TXN,0.033446,0.067953
ADP,0.029817,0.031749
GOOG,0.027813,0.066218
ORCL,0.023743,0.013834
BSX,0.024196,0.011411
UNH,0.038742,0.035452
TMUS,0.027851,0.012886
SYK,0.031048,0.033849
GS,0.025461,0.029023


In [50]:
optimized_df_portfolio = []

for p, weights in optimized_portfolios.items():
    p_stocks = weights.index.tolist()
    
    for ticker in p_stocks:
        p_holdings = df_portfolio[df_portfolio['Portfolio'] == p].set_index('Symbol')['Holding']
        p_t_value = (start_prices[p_stocks] * p_holdings).sum()
        
        new_holding = weights[ticker] * p_t_value / start_prices[ticker]
        
        optimized_df_portfolio.append({
            'Portfolio': p,
            'Symbol': ticker,
            'Holding': new_holding
        })

optimized_df_portfolio = pd.DataFrame(optimized_df_portfolio)

part2_results = run_attribution(realized_returns, realized_spy, last_date, start_prices, optimized_df_portfolio, betas)


                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373  0.011515   0.283866
1  Return Attribution  0.269944  0.013922   0.283866
2     Vol Attribution  0.008035 -0.000501   0.007535

A portfolio attribution:
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373  0.009659   0.288602
1  Return Attribution  0.276657  0.011945   0.288602
2     Vol Attribution  0.007980  0.000035   0.008014

B portfolio attribution:
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373 -0.004927   0.257900
1  Return Attribution  0.262293 -0.004393   0.257900
2     Vol Attribution  0.007451 -0.000099   0.007352

C portfolio attribution:
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373  0.031075   0.305896
1  Return Attribution  0.270254  0.035641   0.305896
2     Vol Attribution  0.007652  0.000553   0.008205


/var/folders/t0/g3jjcv090sj4mcczh5q9hw040000gn/T/ipykernel_60531/4223567146.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.006285537998866152' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  w[ticker] = start_prices[ticker] * portfolio_holdings[ticker] / t_value
/var/folders/t0/g3jjcv090sj4mcczh5q9hw040000gn/T/ipykernel_60531/4223567146.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01795470327232024' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  p_w[ticker] = start_prices[ticker] * p_holdings[ticker] / p_t_value
/var/folders/t0/g3jjcv090sj4mcczh5q9hw040000gn/T/ipykernel_60531/4223567146.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.02

In [87]:
print(attribution_results['Total'])

                Value  SPY(systematic risk)     Alpha  Portfolio
0         TotalReturn              0.261373 -0.035969   0.204731
1  Return Attribution              0.244039 -0.039309   0.204731
2     Vol Attribution              0.007207 -0.000131   0.007076


In [108]:

# compare results between part1 and part2
def calculate_metrics(attribution_df):

    return {
        'Total Return': attribution_df.loc[0, 'Portfolio'],
        'Systematic Return(SPY)': attribution_df.loc[1, 'SPY'],
        'Specific Return(Alpha)': attribution_df.loc[1, 'Alpha'],
        'Portfolio Volatility': attribution_df.loc[2, 'Portfolio'],
        'Systematic Volatility': attribution_df.loc[2, 'SPY'],
        'Specific Volatility': attribution_df.loc[2, 'Alpha']
    }

original_metrics = calculate_metrics(attribution_results['Total'])
optimized_metrics = calculate_metrics(part2_results['Total'])

metrics_comparison = pd.DataFrame({
    'Original Portfolio': original_metrics,
    'Optimized Portfolio': optimized_metrics,
    'Difference': {k: optimized_metrics[k] - original_metrics[k] for k in original_metrics}
})

print(metrics_comparison)


                        Original Portfolio  Optimized Portfolio  Difference
Total Return                      0.204731             0.283866    0.079135
Systematic Return(SPY)            0.244039             0.269944    0.025905
Specific Return(Alpha)           -0.039309             0.013922    0.053231
Portfolio Volatility              0.007076             0.007535    0.000459
Systematic Volatility             0.007207             0.008035    0.000829
Specific Volatility              -0.000131            -0.000501   -0.000370


In [112]:
# compare specific portfolio results
for p in portfolios:
    original_p_result = next(res for res in attribution_results['Portfolios'] if res['Portfolio'] == p)['Attribution']
    optimized_p_result = next(res for res in part2_results['Portfolios'] if res['Portfolio'] == p)['Attribution']
    
    original_p_metrics = calculate_metrics(original_p_result)
    optimized_p_metrics = calculate_metrics(optimized_p_result)
    
    p_metrics_comparison = pd.DataFrame({
        'Original Portfolio': original_p_metrics,
        'Optimized Portfolio': optimized_p_metrics,
        'Difference': {k: optimized_p_metrics[k] - original_p_metrics[k] for k in original_p_metrics}
    })
    
    print(f"\nportfolio {p} comparison")
    print(p_metrics_comparison)


portfolio A comparison
                        Original Portfolio  Optimized Portfolio  Difference
Total Return                      0.136642             0.288602    0.151960
Systematic Return(SPY)            0.242621             0.276657    0.034036
Specific Return(Alpha)           -0.105980             0.011945    0.117924
Portfolio Volatility              0.007404             0.008014    0.000611
Systematic Volatility             0.007056             0.007980    0.000924
Specific Volatility               0.000348             0.000035   -0.000313

portfolio B comparison
                        Original Portfolio  Optimized Portfolio  Difference
Total Return                      0.203526             0.257900    0.054374
Systematic Return(SPY)            0.234259             0.262293    0.028034
Specific Return(Alpha)           -0.030733            -0.004393    0.026340
Portfolio Volatility              0.006854             0.007352    0.000498
Systematic Volatility             0.0064

In [113]:
# compare the optimal weights and idiosyncratic risk for each stocks
original_weights = {}
for ticker in stocks:
    holdings = df_portfolio[df_portfolio['Symbol'] == ticker]['Holding'].sum()
    original_weights[ticker] = holdings * start_prices[ticker] / (start_prices * df_portfolio.set_index('Symbol')['Holding']).sum()

optimized_weights = {}
for ticker in stocks:
    holdings_data = optimized_df_portfolio[optimized_df_portfolio['Symbol'] == ticker]
    if not holdings_data.empty:
        holdings = holdings_data['Holding'].sum()
        optimized_weights[ticker] = holdings * start_prices[ticker] / (start_prices * optimized_df_portfolio.set_index('Symbol')['Holding']).sum()
    else:
        optimized_weights[ticker] = 0

expected_idiosyncratic_risk = {}
for ticker in stocks:
    if ticker == "SPY":
        expected_idiosyncratic_risk[ticker] = 0
    else:
        expected_idiosyncratic_risk[ticker] = np.sqrt(capm_params[ticker]['residuals'])

realized_idiosyncratic_risk = {}
for ticker in stocks:
    if ticker == "SPY":
        realized_idiosyncratic_risk[ticker] = 0
    else:
        excess_returns = hold_returns[ticker] - hold_rf['rf']
        market_excess_returns = hold_returns['SPY'] - hold_rf['rf']
        beta = capm_params[ticker]['beta']
        residuals = excess_returns - beta * market_excess_returns
        realized_idiosyncratic_risk[ticker] = residuals.std()

idiosyncratic_risk_comparison = pd.DataFrame({
    'Original Weight': pd.Series(original_weights),
    'Optimized Weight': pd.Series(optimized_weights),
    'Expected Idiosyncratic Risk': pd.Series(expected_idiosyncratic_risk),
    'Realized Idiosyncratic Risk': pd.Series(realized_idiosyncratic_risk),
    'Difference': {k: realized_idiosyncratic_risk[k] - expected_idiosyncratic_risk[k] for k in expected_idiosyncratic_risk}
})

print(idiosyncratic_risk_comparison)


      Original Weight  Optimized Weight  Expected Idiosyncratic Risk  \
WFC          0.008068          0.006286                     0.014745   
ETN          0.008456          0.013360                     0.013937   
AMZN         0.008282          0.031918                     0.016538   
QCOM         0.010756          0.003815                     0.015578   
LMT          0.011059          0.009664                     0.011105   
...               ...               ...                          ...   
MSFT         0.010636          0.034650                     0.012555   
PEP          0.013260          0.003147                     0.008976   
CB           0.010042          0.017261                     0.012310   
PANW         0.009783          0.002006                     0.022179   
BLK          0.009395          0.009646                     0.009458   

      Realized Idiosyncratic Risk  Difference  
WFC                      0.016993    0.002249  
ETN                      0.012814   -0.

**Part3**

***See doucument***

**Part4**

In [126]:
from scipy.stats import norminvgauss
#normal distribution
def normal_log_likelihood(params, data):
    mu, sigma = params
    if sigma <= 0:
        return np.inf
    return -np.sum(norm.logpdf(data, loc=mu, scale=sigma))

#Generalized t distribution
def gen_t_log_likelihood(params, data):
    mu, sigma, nu = params
    if sigma <= 0 or nu <= 0:
        return np.inf
    const = gammaln((nu + 1) / 2) - gammaln(nu / 2) - 0.5 * np.log(nu * np.pi * sigma**2)
    log_pdf = const - (nu + 1) / 2 * np.log(1 + (data - mu)**2 / (nu * sigma**2))
    return -np.sum(log_pdf)

# NIG distribution
# def nig_pdf(x, alpha, beta, mu, delta):
#     gamma = np.sqrt(np.clip(alpha**2 - beta**2, 1e-10, None))
#     z = (x - mu) / np.clip(delta, 1e-10, None)
#     bessel_arg = np.clip(alpha * delta * np.sqrt(1 + z**2), 1e-10, 700)
#     pdf = (alpha * delta / np.pi) * np.exp(delta * gamma + beta * (x - mu)) * k1e(bessel_arg) / np.sqrt(1 + z**2)
#     return np.clip(pdf, 1e-10, None)

# def nig_log_pdf(x, alpha, beta, mu, delta):
#     diff = np.clip(alpha**2 - beta**2, 1e-10, None)
#     gamma = np.sqrt(diff)
#     z = (x - mu) / delta
#     bessel_arg = alpha * delta * np.sqrt(1 + z**2)
#     log_pdf = np.log(alpha * delta / np.pi) + delta * gamma + beta * (x - mu) + np.log(k1e(bessel_arg)) - 0.5 * np.log(1 + z**2)
#     return log_pdf

# def nig_cdf(x, alpha, beta, mu, delta):
#     cdf_value, err = quad(lambda t: nig_pdf(t, alpha, beta, mu, delta), -np.inf, x)
#     return cdf_value

# def nig_log_likelihood(params, data):
#     alpha, beta, mu, delta = params
#     if alpha <= 0 or delta <= 0 or np.abs(beta) >= alpha:
#         return 1e10
#     return -np.sum(nig_log_pdf(data, alpha, beta, mu, delta))

# Skew Normal distribution
def skew_normal_pdf(x, xi, omega, alpha):
    z = (x - xi) / omega
    return (2 / omega) * norm.pdf(z) * norm.cdf(alpha * z)

def skew_normal_log_likelihood(params, data):
    xi, omega, alpha = params
    if omega <= 0:
        return np.inf
    return -np.sum(np.log(skew_normal_pdf(data, xi, omega, alpha)))

In [128]:
# Fit the distributions
# def nig_initial_params(data):
#     mean = np.mean(data)
#     var = np.clip(np.var(data), 1e-6, None)
#     skew = pd.Series(data).skew()

#     mad = np.median(np.abs(data - np.median(data)))
#     delta_init = np.clip(mad if mad > 0 else np.sqrt(var), 1e-3, 1e3)
    
#     alpha_init = np.clip(1.0 / (delta_init + 1e-6), 1e-3, 100)
#     beta_init_raw = skew / (3 * (delta_init + 1e-6))
#     beta_init = np.clip(beta_init_raw, -0.8 * alpha_init, 0.8 * alpha_init)
    
#     mu_init = mean - delta_init * beta_init / np.sqrt(np.clip(alpha_init**2 - beta_init**2, 1e-10, None))
#     return [alpha_init, beta_init, mu_init, delta_init]

# def fit_nig(data):
#     initial_params = nig_initial_params(data)
#     alpha_init, beta_init, mu_init, delta_init = initial_params
#     bounds = [
#         (max(1e-3, 0.5 * alpha_init), 1.5 * alpha_init),
#         (-0.9 * alpha_init, 0.9 * alpha_init),
#         (np.percentile(data, 1), np.percentile(data, 99)),
#         (max(1e-3, 0.5 * delta_init), 1.5 * delta_init)
#     ]
#     result = minimize(nig_log_likelihood, initial_params, args=(data,),
#                       method='L-BFGS-B', bounds=bounds)
#     alpha, beta, mu, delta = result.x
#     if alpha <= 0 or delta <= 0 or np.abs(beta) >= alpha:
#         return [np.nan] * 4 
#     return result.x

# def fit_nig_robust(data):
#     result = fit_nig(data)
#     if not np.isnan(result[0]):
#         return result
#     bounds = [
#         (1e-3, 100),      # α: Much tighter upper bound
#         (-50, 50),        # β: Reasonable range
#         (np.min(data), np.max(data)),  # μ: Within data range
#         (1e-3, 10)        # δ: Much tighter upper bound
#     ]
#     result_global = differential_evolution(
#         lambda params: nig_log_likelihood(params, data),
#         bounds=bounds,
#         maxiter=100
#     )
#     return result_global.x
def fit_nig(data):
    try:
        alpha, beta, mu, delta = norminvgauss.fit(data)
        if alpha <= 0 or delta <= 0 or np.abs(beta) >= alpha:
            return [np.nan] * 4
        return [alpha, beta, mu, delta]
    except Exception as e:
        print(f"fit NIG error: {e}")
        return [np.nan] * 4
    
def fit_distribution(data, dist_name):
    if dist_name == "Normal":
        mu = np.mean(data)
        sigma = np.std(data)
        params = [mu, sigma]
    elif dist_name == "GenT":
        initial_guess = [np.mean(data), np.std(data), 5.0]
        result = minimize(gen_t_log_likelihood, initial_guess, args=(data,),
                          method='L-BFGS-B', bounds=[(None, None), (1e-5, None), (1e-5, None)])
        params = result.x
    elif dist_name == "NIG":
        params = fit_nig(data)
    elif dist_name == "SkewNormal":
        initial_guess = [np.mean(data), np.std(data), 0.0]
        result = minimize(skew_normal_log_likelihood, initial_guess, args=(data,),
                          method='L-BFGS-B', bounds=[(None, None), (1e-5, None), (None, None)])
        params = result.x
    else:
        params = None
    return params

In [129]:
# Determine the best distribution

def calculate_aic(data, params, dist_name):
    n_params = len(params)
    if dist_name == "Normal":
        log_likelihood = -normal_log_likelihood(params, data)
    elif dist_name == "GenT":
        log_likelihood = -gen_t_log_likelihood(params, data)
    elif dist_name == "NIG":
        # log_likelihood = -nig_log_likelihood(params, data)
        alpha, beta, mu, delta = params
        log_likelihood = np.sum(norminvgauss.logpdf(data, alpha, beta, loc=mu, scale=delta))
    elif dist_name == "SkewNormal":
        log_likelihood = -skew_normal_log_likelihood(params, data)
    else:
        log_likelihood = -np.inf
    return 2 * n_params - 2 * log_likelihood


best_params = {}

for symbol in train_returns.columns:
    data = train_returns[symbol].dropna().values
    aic_scores = {}
    params_dict = {}
    for dist in ["Normal", "GenT", "NIG", "SkewNormal"]:
        params = fit_distribution(data, dist)
        aic = calculate_aic(data, params, dist)
        aic_scores[dist] = aic
        params_dict[dist] = params
    best_model = min(aic_scores, key=aic_scores.get)
    best_params[symbol] = {
        "Best_Model": best_model,
        "Parameters": params_dict[best_model]
    }

best_params_df = pd.DataFrame([
    {"Symbol": symbol, "Best_Model": info["Best_Model"], "Parameters": info["Parameters"]}
    for symbol, info in best_params.items()
])

display(best_params_df)
# pd.set_option('display.max_colwidth', None)
# print(best_params_df['Parameters'])

/var/folders/t0/g3jjcv090sj4mcczh5q9hw040000gn/T/ipykernel_60531/809326607.py:53: RuntimeWarning: divide by zero encountered in log
  return -np.sum(np.log(skew_normal_pdf(data, xi, omega, alpha)))
/Users/yanyiwei/Documents/FinTech545/venv/lib/python3.12/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


,Symbol,Best_Model,Parameters
0,SPY,Normal,"[0.0009849959688576436, 0.008230265429090224]"
1,AAPL,GenT,"[0.001760002237799804, 0.010701217623969377, 7..."
2,NVDA,GenT,"[0.003606099947648441, 0.021823711275835114, 5..."
3,MSFT,GenT,"[0.0016598623360141202, 0.012845638087603707, ..."
4,AMZN,GenT,"[0.0021398343483106886, 0.016486799520298173, ..."
...,...,...,...
95,KKR,GenT,"[0.002555003948757106, 0.01616928163457702, 5...."
96,MU,NIG,"[1.45304422688949, 0.5480757220672373, -0.0077..."
97,PLD,GenT,"[0.0009519211835215854, 0.01339075971165784, 5..."
98,LRCX,NIG,"[1.625477094139498, 0.555045323080846, -0.0064..."


In [130]:
# def nig_inv_cdf(u, alpha, beta, mu, delta, num_points=1000, ext_range=10):

#     x_min = mu - ext_range * delta
#     x_max = mu + ext_range * delta
#     x_grid = np.linspace(x_min, x_max, num_points)
    
#     cdf_values = np.array([nig_cdf(x, alpha, beta, mu, delta) for x in x_grid])
    
#     inv_cdf_func = interp1d(
#         cdf_values, x_grid, 
#         kind='linear', 
#         bounds_error=False, 
#         fill_value=(x_min, x_max))
    
#     u = np.clip(u, 1e-6, 1-1e-6)
#     quantiles = inv_cdf_func(u)
    
#     mask_low = (u < cdf_values[0])
#     mask_high = (u > cdf_values[-1])
    
#     if np.any(mask_low) or np.any(mask_high):
#         gamma = np.sqrt(alpha**2 - beta**2)
#         approx_mu = mu + (beta * delta) / gamma
#         approx_sigma = delta / gamma
        
#         quantiles[mask_low] = approx_mu + approx_sigma * norm.ppf(u[mask_low])
#         quantiles[mask_high] = approx_mu + approx_sigma * norm.ppf(u[mask_high])
    
#     return quantiles


corr_matrix = train_returns.corr()
num_assets = len(train_returns.columns)
mv_normal = multivariate_normal(mean=np.zeros(num_assets), cov=corr_matrix)  
samples = mv_normal.rvs(10000)  
uniform_samples = norm.cdf(samples)  

simulated_returns = np.zeros_like(uniform_samples)
# nig_cache = {}

for i, symbol in enumerate(train_returns.columns):
    model_info = best_params[symbol]
    best_model = model_info["Best_Model"]
    params = model_info["Parameters"]
    
    if best_model == "NIG":
        alpha, beta, mu, delta = params
        
        if (alpha <= np.abs(beta)) or (delta <= 0) or np.isnan(params).any():
            gamma = np.sqrt(alpha**2 - beta**2) if alpha > np.abs(beta) else 1e-6
            approx_mu = mu + (beta * delta) / gamma
            approx_sigma = np.clip(delta / gamma, 1e-6, 1e3)
            simulated_returns[:, i] = approx_mu + approx_sigma * norm.ppf(uniform_samples[:, i])
            continue
        # if (alpha, beta, mu, delta) not in nig_cache:
        #     x_min = mu - 15*delta
        #     x_max = mu + 15*delta
        #     x_grid = np.linspace(x_min, x_max, 2000)
        #     cdf_grid = np.array([nig_cdf(x, *params) for x in x_grid])
            

        #     unique_cdf, unique_idx = np.unique(cdf_grid, return_index=True)
        #     inv_cdf_func = interp1d(
        #         unique_cdf, x_grid[unique_idx],
        #         kind='linear',
        #         bounds_error=False,
        #         fill_value=(x_min, x_max))
            
        #     nig_cache[(alpha, beta, mu, delta)] = inv_cdf_func
        
        # inv_cdf_func = nig_cache[(alpha, beta, mu, delta)]
        # simulated_returns[:, i] = inv_cdf_func(uniform_samples[:, i])
        try:
            u_clipped = np.clip(uniform_samples[:, i], 1e-6, 1-1e-6)
            simulated_returns[:, i] = norminvgauss.ppf(
                u_clipped, 
                a=alpha,
                b=beta,
                loc=mu, 
                scale=delta
            )
        except Exception as e:
            print(f"{symbol} NIG quantile calculate error, try multinormal: {e}")
            gamma = np.sqrt(alpha**2 - beta**2)
            approx_mu = mu + (beta * delta) / gamma
            approx_sigma = delta / gamma
            simulated_returns[:, i] = approx_mu + approx_sigma * norm.ppf(u_clipped)
        
        nan_mask = np.isnan(simulated_returns[:, i])
        if np.any(nan_mask):
            gamma = np.sqrt(alpha**2 - beta**2)
            approx_mu = mu + (beta * delta) / gamma
            approx_sigma = delta / gamma
            # simulated_returns[nan_mask, i] = approx_mu + approx_sigma * norm.ppf(uniform_samples[nan_mask, i])
            simulated_returns[nan_mask, i] = norm.ppf(
                uniform_samples[nan_mask, i], 
                loc=approx_mu, 
                scale=approx_sigma
            )
    elif best_model == "SkewNormal":
        xi, omega, alpha_sn = params
        simulated_returns[:, i] = xi + omega * norm.ppf(uniform_samples[:, i])
    
    else:
        simulated_returns[:, i] = norm.ppf(uniform_samples[:, i], loc=params[0], scale=params[1])


results_copula = []
portfolios = df_portfolio["Portfolio"].unique()

for port in portfolios:
    port_df = df_portfolio[df_portfolio["Portfolio"] == port]
    symbols = port_df["Symbol"].tolist()
    indices = [list(train_returns.columns).index(sym) for sym in symbols if sym in train_returns.columns]
    weights = port_df.set_index("Symbol").loc[symbols, "Holding"].values
    # weights = weights / np.sum(np.abs(weights))
    port_sim_returns = simulated_returns[:, indices]
    port_returns = port_sim_returns @ weights
    port_loss = -port_returns
    var_val = np.percentile(port_loss, 95)
    es_val = np.mean(port_loss[port_loss >= var_val])
    results_copula.append({"Portfolio": port, "VaR_Copula": var_val, "ES_Copula": es_val})


common_symbols = [sym for sym in train_returns.columns if sym in df_portfolio["Symbol"].values]
all_weights = df_portfolio.set_index("Symbol").loc[common_symbols, "Holding"].values
simulated_returns_common = simulated_returns[:, [list(train_returns.columns).index(sym) for sym in common_symbols]]
total_returns = simulated_returns_common @ all_weights
total_loss = -total_returns
var_total = np.percentile(total_loss, 95)
es_total = np.mean(total_loss[total_loss >= var_total])


PLTR NIG quantile calculate error, try multinormal: The function value at x=nan is NaN; solver cannot continue.
SYK NIG quantile calculate error, try multinormal: The function value at x=nan is NaN; solver cannot continue.


In [131]:
results_mvnorm = []
for port in portfolios:
    port_df = df_portfolio[df_portfolio["Portfolio"] == port]
    symbols = port_df["Symbol"].tolist()
    indices = [list(train_returns.columns).index(sym) for sym in symbols if sym in train_returns.columns]
    weights = port_df.set_index("Symbol").loc[symbols, "Holding"].values
    mean_vec = train_returns[symbols].mean().values
    cov_mat = train_returns[symbols].cov().values
    sim_mvnorm = multivariate_normal(mean=mean_vec, cov=cov_mat).rvs(10000)
    if sim_mvnorm.ndim == 1:
        sim_mvnorm = sim_mvnorm.reshape(-1, 1)
    port_returns_mv = sim_mvnorm @ weights
    port_loss_mv = -port_returns_mv
    var_mv = np.percentile(port_loss_mv, 95)
    es_mv = np.mean(port_loss_mv[port_loss_mv >= var_mv])
    results_mvnorm.append({"Portfolio": port, "VaR_MVNorm": var_mv, "ES_MVNorm": es_mv})

mean_all = train_returns[common_symbols].mean().values
cov_all = train_returns[common_symbols].cov().values
sim_mvnorm_all = multivariate_normal(mean=mean_all, cov=cov_all).rvs(10000)
if sim_mvnorm_all.ndim == 1:
    sim_mvnorm_all = sim_mvnorm_all.reshape(-1, 1)
total_returns_mv = sim_mvnorm_all @ all_weights
total_loss_mv = -total_returns_mv
var_total_mv = np.percentile(total_loss_mv, 95)
es_total_mv = np.mean(total_loss_mv[total_loss_mv >= var_total_mv])

df_copula = pd.DataFrame(results_copula)
df_mvnorm = pd.DataFrame(results_mvnorm)

total_result = {
    "Portfolio": "Total",
    "VaR_Copula": var_total,
    "ES_Copula": es_total,
    "VaR_MVNorm": var_total_mv,
    "ES_MVNorm": es_total_mv
}

df_copula = df_copula.merge(df_mvnorm, on="Portfolio", how="outer")
df_total = pd.DataFrame([total_result])
df_all = pd.concat([df_copula, df_total], ignore_index=True)

print(df_all)


  Portfolio  VaR_Copula  ES_Copula  VaR_MVNorm   ES_MVNorm
0         A   31.800449  40.482925   40.865056   52.344551
1         B   20.811029  25.740612   28.633686   36.248001
2         C   25.735931  32.862662   29.174060   37.530340
3     Total   71.963539  90.318627   90.127561  114.295830


***Part5***

In [159]:
def portfolio_ES(weights, sim_returns, alpha=95):
    port_returns = np.dot(sim_returns, weights)
    losses = -port_returns
    var = np.percentile(losses, alpha)
    es = np.mean(losses[losses >= var])
    return es

def component_ES(weights, sim_returns, alpha=95, eps=1e-4):
    base_es = portfolio_ES(weights, sim_returns, alpha)
    components = np.zeros_like(weights)
    
    for i in range(len(weights)):
        old_weight = weights[i]
        perturbed = weights.copy()
        perturbed[i] += eps
        # 为保持和为1，从其它分量里扣 eps/(n−1)
        for j in range(len(weights)):
            if j != i:
                perturbed[j] -= eps / (len(weights) - 1)
        perturbed = np.maximum(perturbed, 0)
        perturbed /= np.sum(perturbed)
        # components[i] = old_weight * (portfolio_ES(perturbed, sim_returns, alpha) - base_es) / eps
        dES = (portfolio_ES(perturbed, sim_returns, alpha) - base_es) / eps
        components[i] = dES
    return components

def risk_parity_objective(weights, sim_returns, alpha=95, eps=1e-4):
    components = component_ES(weights, sim_returns, alpha, eps)
    # target_contrib = np.mean(components)
    # return 1e5 * np.sum((components - target_contrib)**2)
    rc = weights * components    # weight * dES
    target = np.mean(rc)
    obj = np.sum((rc - target)**2)
    return obj


def constraint_sum_to_one(weights):
    return np.sum(weights) - 1

def optimize_risk_parity(sim_returns, alpha=95, eps=1e-4):
    
    n_assets = sim_returns.shape[1]
    initial_weights = np.ones(n_assets) / n_assets
    constraints = [{'type': 'eq', 'fun': constraint_sum_to_one}]
    bounds = [(0.0, 1.0) for _ in range(n_assets)]
    result = minimize(
        risk_parity_objective,
        initial_weights,
        args=(sim_returns, alpha, eps),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'ftol': 1e-10, 'maxiter': 500, 'disp': True}
    )
    
    final_weights = result.x
    components = component_ES(final_weights, sim_returns, alpha, eps)
    target = np.mean(components)
    deviations = np.abs(components / target - 1)
    max_dev = np.max(deviations)
    
    print(f"优化完成: 目标函数值 = {result.fun:.6e}, 迭代次数 = {result.nit}")
    print(f"最大风险贡献偏差: {max_dev:.4f}")
    
    if max_dev > 0.1:
        print("警告: 风险贡献不完全均衡")
    
    return final_weights, result.fun

portfolios = df_portfolio['Portfolio'].unique()
risk_parity_portfolios = {}
stock_indices = {}

for p in portfolios:
    print(f"\n开始优化 {p} 组合...")
    
    # 获取子投资组合的股票
    p_stocks = df_portfolio[df_portfolio['Portfolio'] == p]['Symbol'].unique().tolist()
    
    # 找到这些股票在simulated_returns中的索引
    valid_stocks = []
    valid_indices = []
    for s in p_stocks:
        if s in train_returns.columns:
            idx = list(train_returns.columns).index(s)
            valid_stocks.append(s)
            valid_indices.append(idx)
    
    if len(valid_stocks) == 0:
        print(f"警告: {p} 组合中没有有效股票，跳过优化")
        continue
    
    stock_indices[p] = valid_indices
    
    # 提取子投资组合的模拟收益率
    p_sim_returns = simulated_returns[:, valid_indices]
    
    # 优化风险平价权重
    p_weights, _ = optimize_risk_parity(p_sim_returns, alpha=95, eps=1e-4)
    
    # 保存权重
    risk_parity_portfolios[p] = dict(zip(valid_stocks, p_weights))
    
    # 打印结果
    result_df = pd.DataFrame({
        'Symbol': valid_stocks,
        'Weight': p_weights,
        'ES Contribution': component_ES(p_weights, p_sim_returns)
    })
    print(f"{p} 组合的风险平价权重:")
    print(result_df)


开始优化 A 组合...
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.0162295763937674e-07
            Iterations: 26
            Function evaluations: 902
            Gradient evaluations: 26
优化完成: 目标函数值 = 3.016230e-07, 迭代次数 = 26
最大风险贡献偏差: 36.2008
警告: 风险贡献不完全均衡
A 组合的风险平价权重:
   Symbol        Weight  ES Contribution
0     WFC  4.170590e-02         0.003740
1     ETN  2.627103e-02         0.005638
2    AMZN  5.301321e-02        -0.000180
3    QCOM  2.430809e-02         0.005781
4     LMT  2.570265e-03        -0.009645
5      KO  3.382079e-03        -0.010009
6     JNJ  1.125529e-03        -0.010115
7    ISRG  5.247546e-02         0.001445
8     XOM  5.261338e-02         0.000731
9     MDT  3.932439e-02        -0.004136
10    DHR  5.186910e-02         0.000503
11    PLD  4.395736e-02         0.003652
12     BA  5.270111e-02         0.000245
13     PG  2.676703e-14        -0.009993
14    MRK  5.107239e-03        -0.009925
15    AMD  1.213883e-03         

In [160]:
import numpy as np
from scipy.optimize import minimize

# --- 1) 组合 ES 计算 ---
def portfolio_ES(weights, sim_returns, alpha=95):
    """
    计算组合 ES：损失 = -收益，VaR = alpha 分位数，ES = VaR 以上损失平均
    """
    port_ret = sim_returns.dot(weights)
    losses = -port_ret
    var = np.percentile(losses, alpha)
    return np.mean(losses[losses >= var])

# --- 2) ∂ES/∂w 差分估计 (正向差分 + 等量扣减) ---
def es_sensitivity(weights, sim_returns, alpha=95, eps=1e-4):
    """
    返回 ∂ES/∂w 向量，长度 = n_assets
    只对 w[i] + eps，并从其它 w[j] 等量扣 eps/(n-1)，保持 ∑w=1。
    """
    base_es = portfolio_ES(weights, sim_returns, alpha)
    n = len(weights)
    grad = np.zeros(n)
    for i in range(n):
        pert = weights.copy()
        pert[i] += eps
        # 从其它分量等量扣除
        pert[np.arange(n) != i] -= eps / (n - 1)
        # **不再 clamp/renorm**
        grad[i] = (portfolio_ES(pert, sim_returns, alpha) - base_es) / eps
    return grad

# --- 3) 风险平价目标函数 (用风险贡献) ---
def risk_parity_obj(weights, sim_returns, alpha=95, eps=1e-4):
    """
    RC_i = w_i * dES/dw_i
    最小化 SSE = Σ (RC_i - mean(RC))^2
    """
    grad = es_sensitivity(weights, sim_returns, alpha, eps)
    rc   = weights * grad
    target = rc.mean()
    sse = np.sum((rc - target)**2)
    return 1e5 * sse  # 放大以增强数值稳定性

# --- 4) 约束与优化器入口 ---
def constraint_sum(weights):
    return np.sum(weights) - 1

def optimize_risk_parity(sim_returns, alpha=95, eps=1e-4):
    n = sim_returns.shape[1]
    w0 = np.ones(n) / n
    cons = ({'type':'eq','fun':constraint_sum},)
    bounds = [(0,1)] * n
    res = minimize(
        risk_parity_obj, w0,
        args=(sim_returns, alpha, eps),
        method='SLSQP',
        bounds=bounds,
        constraints=cons,
        options={'ftol':1e-10,'maxiter':500,'disp':False}
    )
    w_opt = res.x
    # 诊断输出
    grad = es_sensitivity(w_opt, sim_returns, alpha, eps)
    rc   = w_opt * grad
    dev  = np.max(np.abs(rc/rc.mean() - 1))
    print(f"  收敛状态: success={res.success}, nit={res.nit}, max_dev={dev:.2%}")
    return w_opt

# --- 5) 对每个子组合做风险平价 ---
risk_parity_df = []
for p in df_portfolio['Portfolio'].unique():
    print(f"\n== 优化子组合 {p} ==")
    sub = df_portfolio.query("Portfolio==@p")
    symbols = list(sub['Symbol'])
    # 找到模拟中对应的列
    idxs = [list(train_returns.columns).index(s) for s in symbols]
    sim_sub = simulated_returns[:, idxs]
    # 优化
    w_sub = optimize_risk_parity(sim_sub, alpha=95, eps=1e-4)
    # 打印结果
    df_w = pd.DataFrame({
        'Symbol': symbols,
        'Weight': w_sub,
        'RC': w_sub * es_sensitivity(w_sub, sim_sub, 95, 1e-4)
    })
    print(df_w.to_string(index=False))
    # 换算为新的持仓数量
    holdings = sub.set_index('Symbol')['Holding']
    total_value = sum(start_prices[s] * holdings[s] for s in symbols)
    for sym, w in zip(symbols, w_sub):
        qty = w * total_value / start_prices[sym]
        risk_parity_df.append({
            'Portfolio': p,
            'Symbol': sym,
            'Holding': qty
        })

risk_parity_df = pd.DataFrame(risk_parity_df)
print("\n== 新的风险平价持仓 ==")
print(risk_parity_df)

# -----------------------------------------
# 后续你可以将 risk_parity_df 传入之前的 run_attribution，
# 与 CAPM beta 配合，完成 Part5 的归因分析。
# -----------------------------------------


== 优化子组合 A ==
  收敛状态: success=True, nit=108, max_dev=131.77%
Symbol       Weight            RC
   WFC 3.496193e-02  1.344335e-04
   ETN 2.040007e-02  1.069075e-04
  AMZN 7.838869e-02  1.293936e-04
  QCOM 1.297729e-02  8.049927e-05
   LMT 2.142907e-08 -1.968431e-10
    KO 1.446378e-17 -1.443562e-19
   JNJ 6.199346e-16 -6.210068e-18
  ISRG 7.475734e-02  1.137543e-04
   XOM 8.143799e-02  1.098263e-04
   MDT 7.441230e-09 -3.504363e-11
   DHR 8.221948e-02  1.217571e-04
   PLD 2.535540e-02  9.067602e-05
    BA 8.953519e-02  1.266531e-04
    PG 4.516917e-17 -4.576079e-19
   MRK 2.387270e-15 -2.395728e-17
   AMD 1.060530e-02  7.952778e-05
    BX 1.296740e-02  9.204564e-05
    PM 2.818538e-09 -1.981207e-11
  SCHW 3.300578e-02  1.062620e-04
    VZ 1.032680e-09 -9.273096e-12
   COP 9.754469e-02  1.507129e-04
   ADI 4.648646e-02  9.798601e-05
   BAC 9.151286e-03  1.001566e-04
   NOW 6.993003e-03  7.461933e-05
   TMO 7.768400e-02  1.106324e-04
   CVX 5.146808e-03 -8.109558e-06
  ANET 1.132455e-03 

In [153]:
risk_parity_df = []

for p, weights_dict in risk_parity_portfolios.items():
    # 获取子投资组合的现有持仓价值
    p_holdings = df_portfolio[df_portfolio['Portfolio'] == p].set_index('Symbol')['Holding']
    p_stocks = list(weights_dict.keys())
    p_t_value = sum(start_prices[s] * p_holdings[s] for s in p_stocks if s in p_holdings.index)
    
    for ticker, weight in weights_dict.items():
        # 新持股量 = 权重 * 总价值 / 股票价格
        if ticker in start_prices.index:
            new_holding = weight * p_t_value / start_prices[ticker]
            risk_parity_df.append({
                'Portfolio': p,
                'Symbol': ticker,
                'Holding': new_holding
            })

risk_parity_df = pd.DataFrame(risk_parity_df)
print("\n风险平价投资组合构建完成:")
print(risk_parity_df.groupby('Portfolio')['Holding'].count())
print(risk_parity_df)


风险平价投资组合构建完成:
Portfolio
A    33
B    33
C    33
Name: Holding, dtype: int64
   Portfolio Symbol     Holding
0          A    WFC   42.153144
1          A    ETN    0.000001
2          A   AMZN  182.418982
3          A   QCOM    0.000002
4          A    LMT    0.000002
..       ...    ...         ...
94         C   MSFT   81.396802
95         C    PEP    0.000001
96         C     CB    0.000001
97         C   PANW    0.000002
98         C    BLK   36.598155

[99 rows x 3 columns]


In [154]:
realized_returns = hold_returns[stocks]
realized_spy = pd.DataFrame({'SPY': hold_returns['SPY']})
last_date = train_returns.index[-1]

# 运行风险平价投资组合的归因分析
print("\n===== 运行风险平价投资组合的归因分析 =====")
part5_results = run_attribution(realized_returns, realized_spy, last_date, start_prices, risk_parity_df, betas)



===== 运行风险平价投资组合的归因分析 =====
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373 -0.022666   0.251373
1  Return Attribution  0.275937 -0.024564   0.251373
2     Vol Attribution  0.007902  0.000046   0.007948

A portfolio attribution:
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373 -0.050771   0.212265
1  Return Attribution  0.268812 -0.056547   0.212265
2     Vol Attribution  0.007112  0.001774   0.008886

B portfolio attribution:
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373 -0.045105   0.210054
1  Return Attribution  0.259187 -0.049133   0.210054
2     Vol Attribution  0.006882  0.000642   0.007524

C portfolio attribution:
                Value       SPY     Alpha  Portfolio
0         TotalReturn  0.261373  0.031281   0.337925
1  Return Attribution  0.301403  0.036522   0.337925
2     Vol Attribution  0.007903  0.001438   0.009341


/var/folders/t0/g3jjcv090sj4mcczh5q9hw040000gn/T/ipykernel_60531/4223567146.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0023951348892619754' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  w[ticker] = start_prices[ticker] * portfolio_holdings[ticker] / t_value
/var/folders/t0/g3jjcv090sj4mcczh5q9hw040000gn/T/ipykernel_60531/4223567146.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.006841727095684163' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  p_w[ticker] = start_prices[ticker] * p_holdings[ticker] / p_t_value
/var/folders/t0/g3jjcv090sj4mcczh5q9hw040000gn/T/ipykernel_60531/4223567146.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.

In [21]:
# ------------------------------
# 2. 利用新风险平价权重进行持仓期归因
# ------------------------------
# Part 1 中，我们已利用 CAPM 回归得到各股票的 beta 参数，存储在 capm_params 字典中，
# 格式为：capm_params[ticker] = { 'alpha': ..., 'beta': ..., 'residuals': ... }
#
# 在此我们采用新组合权重（opt_weights）重新构造持仓期的组合，
# 然后利用 CAPM（假设0α）把组合累计超额收益分解为系统性和特质成分。

# 假设 train_returns, hold_returns, hold_rf 已在 Part1 中定义，
# df_prices 与 df_rf 等也已处理过。train_returns 的列顺序与风险平价权重对应（例如所有股票的顺序）。
# 下面 new_weights 为全市场新的风险平价权重；
# 若需分子组合，可以在 df_portfolio 中根据 Symbol 将其“子组合”划分，再利用 opt_weights 中对应股票构造局部权重。

# 本例中演示对整体组合的归因（也可按子组合分别计算）。

# 取所有股票（假设其顺序与 train_returns.columns 相同）
all_stock_symbols = list(train_returns.columns)
# 风险平价组合（全市场）新权重 opt_weights，对应股票列表 all_stock_symbols

# 利用新权重构造持仓期组合日收益（假设持仓期收益 hold_returns 包含所有股票）
# 此处我们采用固定权重（即 opt_weights 不随时间变动）
new_port_returns = hold_returns[all_stock_symbols].dot(opt_weights)
# 新组合超额收益 = 组合日收益 - 持仓期风险自由利率
# 注意 hold_rf 的形状需与 hold_returns 对应（风险自由率逐日）
new_port_excess = new_port_returns - hold_rf['rf'].values

# 计算新组合累计超额收益（采用连续复利或算数累计，本例采用算数累计）
def cumulative_return(returns):
    return np.prod(1 + returns) - 1

new_port_cum_excess = cumulative_return(new_port_excess)
print("New risk parity portfolio cumulative excess return:", new_port_cum_excess)

# 利用之前 CAPM 拟合的 beta（capm_params）对每只股票分别计算其持仓期间累计超额收益
stock_attribution = {}
for ticker in all_stock_symbols:
    stock_hold = hold_returns[ticker].dropna()  # 持仓期股票收益
    stock_rf = hold_rf['rf'].loc[stock_hold.index]
    stock_excess = stock_hold - stock_rf
    stock_cum_excess = cumulative_return(stock_excess)
    # 若为 SPY 则 beta=1
    if ticker == "SPY":
        beta = 1
    else:
        beta = capm_params[ticker]['beta']
    # 系统性部分
    sys_return = beta * cumulative_return(hold_returns['SPY'] - hold_rf['rf'])
    # 剩余部分归为特质风险
    idio_return = stock_cum_excess - sys_return
    stock_attribution[ticker] = {
        'cum_excess_return': stock_cum_excess,
        'beta': beta,
        'sys_return': sys_return,
        'idio_return': idio_return
    }

df_stock_attr = pd.DataFrame.from_dict(stock_attribution, orient='index')
print("股票层面的累计超额收益归因（CAPM分解）：")
print(df_stock_attr)

# 接下来，对新组合进行归因：
# 组合系统性回报 = Σ (new_weight × 股票 beta) * 市场累计超额收益
# 组合累计超额回报 = Σ (new_weight × 股票累计超额回报)
# 故，组合特质部分 = 组合累计超额回报 - 组合系统性回报

# 提取新风险平价权重对应股票的 CAPM参数及累计超额收益
new_weights_series = pd.Series(opt_weights, index=all_stock_symbols)

# 对整体新组合：
comb_sys_return = np.sum(new_weights_series * df_stock_attr['beta']) * cumulative_return(hold_returns['SPY'] - hold_rf['rf'])
comb_cum_return = np.sum(new_weights_series * df_stock_attr['cum_excess_return'])
comb_idio_return = comb_cum_return - comb_sys_return

new_attr = pd.DataFrame({
    'Portfolio': ['Risk Parity (ES)'],
    'cum_excess_return': [comb_cum_return],
    'systematic_return': [comb_sys_return],
    'idiosyncratic_return': [comb_idio_return]
})
print("新风险平价组合的归因：")
print(new_attr)


New risk parity portfolio cumulative excess return: 0.13737025687200877
股票层面的累计超额收益归因（CAPM分解）：
      cum_excess_return      beta  sys_return  idio_return
SPY            0.198781  1.000000    0.198781     0.000000
AAPL           0.207105  1.103753    0.219405    -0.012300
NVDA           1.773573  2.018054    0.401151     1.372422
MSFT           0.075778  1.169683    0.232511    -0.156733
AMZN           0.402303  1.532365    0.304605     0.097698
...                 ...       ...         ...          ...
KKR            0.754982  1.708098    0.339538     0.415445
MU             0.005049  1.356371    0.269621    -0.264572
PLD           -0.222450  1.265703    0.251598    -0.474047
LRCX          -0.061250  1.659387    0.329855    -0.391105
EQIX           0.150437  1.067262    0.212152    -0.061714

[100 rows x 4 columns]
新风险平价组合的归因：
          Portfolio  cum_excess_return  systematic_return  \
0  Risk Parity (ES)            0.13273           0.140778   

   idiosyncratic_return  
0           